In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import os
from keras.preprocessing import image

Using TensorFlow backend.


In [27]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tqdm import tqdm
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense
from keras import optimizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [4]:
data_dir = '/Users/alexeydemyanchuk/Kaggle/DogBreedIdent'

In [5]:
# dataframes for training and testing
labels = pd.read_csv('labels.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [6]:
SEED = 1987
INPUT_SIZE = 64

In [7]:
# print(len(os.listdir(os.path.join(data_dir, 'train'))), len(labels))
# print(len(os.listdir(os.path.join(data_dir, 'test'))), len(sample_submission))

10222 10222
10357 10357


In [8]:
# train/valid split indecise
np.random.seed(seed=SEED)
rnd = np.random.random(len(labels))
train_idx = rnd < 0.8
valid_idx = rnd >= 0.8

In [9]:
# from categorical to one hot 
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(120).index)
labels['target'] = 1
labels['rank'] = labels.groupby('breed').rank()['id']
labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)

In [10]:
y_train = labels_pivot[selected_breed_list].values
ytr = y_train[train_idx]
yv = y_train[valid_idx]

In [11]:
print(ytr.shape, yv.shape)

(8211, 120) (2011, 120)


In [12]:
# helper to read, resize image and convert it to numpy array 
def read_img(img_id, train_or_test, size):
    """Read and resize image.
    # Arguments
        img_id: string
        train_or_test: string 'train' or 'test'.
        size: resize the original image.
    # Returns
        Image as numpy array.
    """
    img = image.load_img(os.path.join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

In [13]:
# process all training images
x_train = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

10222it [00:48, 211.79it/s]

Train Images shape: (10222, 64, 64, 3) size: 125,607,936


In [14]:
# split training data in training and validation sets
Xtr = x_train[train_idx]
Xv = x_train[valid_idx]
print(Xtr.shape, Xv.shape)

(8211, 64, 64, 3) (2011, 64, 64, 3)


In [16]:
# importing pretrained model not including FC layers for imagenet classification
model = VGG16(weights="imagenet", include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [17]:
len(model.layers)

19

In [18]:
# froze layers from pretrained
for layer in model.layers[:19]:
    layer.trainable = False

In [21]:
# connecting custom layers to classify dog breeds
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(ytr.shape[1], activation="softmax")(x)

In [22]:
model_final = Model(inputs=model.input, outputs=predictions)

In [24]:
model_final.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [26]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [29]:
callbacks = [EarlyStopping(), ReduceLROnPlateau()]

In [30]:
model_final.fit(Xtr, ytr, validation_data=(Xv, yv), batch_size=64,
                epochs=50, callbacks=callbacks,verbose=1)

Train on 8211 samples, validate on 2011 samples
Epoch 1/50
8211/8211 [==============================] - 311s - loss: 15.2750 - acc: 0.0208 - val_loss: 14.9360 - val_acc: 0.0323
Epoch 2/50
8211/8211 [==============================] - 309s - loss: 14.2790 - acc: 0.0697 - val_loss: 14.5038 - val_acc: 0.0497
Epoch 3/50
8211/8211 [==============================] - 320s - loss: 13.5777 - acc: 0.1118 - val_loss: 14.3441 - val_acc: 0.0567
Epoch 4/50
8211/8211 [==============================] - 496s - loss: 13.0224 - acc: 0.1477 - val_loss: 14.0580 - val_acc: 0.0681
Epoch 5/50
8211/8211 [==============================] - 566s - loss: 12.5704 - acc: 0.1777 - val_loss: 13.9526 - val_acc: 0.0696
Epoch 6/50
8211/8211 [==============================] - 558s - loss: 12.1944 - acc: 0.1994 - val_loss: 13.7620 - val_acc: 0.0781
Epoch 7/50
8211/8211 [==============================] - 582s - loss: 11.8645 - acc: 0.2268 - val_loss: 13.6090 - val_acc: 0.0835
Epoch 8/50
8211/8211 [===========================

In [ ]:
filepath = os.path.join(data_dir, "vgg16_imgnet_pretrained_50epochs.h5")
model_final.save_weights(filepath)